In [14]:
import os
import time 
import datetime

def print_table(data: dict[str, list[float]], precision: int = 4):
    # Convert to list of keys and find number of rows
    headers = list(data.keys())
    headers = ["index"] + headers        
    n_rows = len(next(iter(data.values())))
    data["index"] = list(range(1, n_rows+1))

    # Format floats to desired precision
    fmt = f"{{:.{precision}f}}"

    # Compute column widths
    col_widths = []
    for h in headers:
        values = data[h]
        max_val_len = max(len(fmt.format(v)) for v in values)
        col_widths.append(max(len(h), max_val_len))

    # Build header row
    header_row = " | ".join(f"{h:>{w}}" for h, w in zip(headers, col_widths))
    separator = "-+-".join("-" * w for w in col_widths)

    # Print table
    print(header_row)
    print(separator)
    for i in range(n_rows):
        row = " | ".join(fmt.format(data[h][i]).rjust(w) for h, w in zip(headers, col_widths))
        print(row)

In [ ]:
def dt_to_intkey(date_str: str, time_str: str) -> float:
    dt = datetime.datetime.strptime(f"{date_str} {time_str}", "%Y_%m_%d %H_%M_%S")
    return dt.timestamp()

In [23]:
data_dir = "../../logs/results"
res_files_unfiltered = [f for f in os.listdir(data_dir) if f.endswith(".log")]
res_files_unfiltered.sort()
# print(*res_files, sep="\n")
dates, times = zip(*(s.split("__")[:2] for s in res_files_unfiltered))
creation_times = {
    # key is date_time string converted to int/float for sorting
    index: dt_to_intkey(date, time)
    for index, (date, time) in enumerate(zip(dates, times))
}
only_after = dt_to_intkey("2025_10_18", "00_00_00")
res_files = [
    res_files_unfiltered[index]
    for index, creation_time in creation_times.items()
    if creation_time >= only_after
]
# print(*res_files, sep="\n")

In [24]:


results_y_co_cg = [{} for _ in range(23)]
results_rgb = [{} for _ in range(23)]

for f in res_files:
    with open(os.path.join(data_dir, f), "r") as infile:
        lines = infile.readlines()
        ycocg = True
        for line in lines:
            if "RGB" in line:
                ycocg = False
            if "Final results after quantization" in line:
                parts = line[len("Final results after quantization:"):].strip().split(", ")
                for part in parts:
                    key, value = part.split(": ")
                    if ycocg:
                        results_y_co_cg[int(f.split("_")[-1][len("kodim"):][:-len(".log")])-1][key] = float(value)
                    else:
                        results_rgb[int(f.split("_")[-1][len("kodim"):][:-len(".log")])-1][key] = float(value)




def list_dict_to_dict_list(lst: list[dict[str, float]]) -> dict[str, list[float]]:
    if not lst:
        return {}
    keys = lst[0].keys()
    dict_list = {key: [] for key in keys}
    for d in lst:
        for key in keys:
            dict_list[key].append(d[key])
    return dict_list

results_y_co_cg = list_dict_to_dict_list(results_y_co_cg)
results_rgb = list_dict_to_dict_list(results_rgb)

print("YCoCg Results:")
print_table(results_y_co_cg)
for key, values in results_y_co_cg.items():
    avg = sum(values) / len(values) if values else 0
    print(f"{key}: {avg:.4f}")

print("RGB Results:")
print_table(results_rgb)
for key, values in results_rgb.items():
    avg = sum(values) / len(values) if values else 0
    print(f"{key}: {avg:.4f}")


YCoCg Results:
  index |   Loss | Rate NN | Rate Latent | Rate Img
--------+--------+---------+-------------+---------
 1.0000 | 2.4217 |  0.0079 |      0.0112 |   2.4025
 2.0000 | 1.8849 |  0.0087 |      0.0012 |   1.8750
 3.0000 | 1.7287 |  0.0046 |      0.0487 |   1.6754
 4.0000 | 1.9818 |  0.0205 |      0.0239 |   1.9374
 5.0000 | 2.4079 |  0.0088 |      0.0011 |   2.3980
 6.0000 | 2.2266 |  0.0267 |      0.0909 |   2.1091
 7.0000 | 1.8009 |  0.0227 |      0.0255 |   1.7527
 8.0000 | 2.4602 |  0.0038 |      0.0250 |   2.4314
 9.0000 | 1.9424 |  0.0042 |      0.0451 |   1.8931
10.0000 | 1.9464 |  0.0039 |      0.0157 |   1.9267
11.0000 | 2.0339 |  0.0045 |      0.0096 |   2.0198
12.0000 | 1.7989 |  0.0041 |      0.0251 |   1.7697
13.0000 | 2.7063 |  0.0256 |      0.1216 |   2.5592
14.0000 | 2.1602 |  0.0071 |      0.0007 |   2.1524
15.0000 | 1.8158 |  0.0091 |      0.0187 |   1.7879
16.0000 | 1.9517 |  0.0279 |      0.0948 |   1.8290
17.0000 | 1.8693 |  0.0062 |      0.0082 |   1.85